In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io
import openpyxl
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import mean_squared_error

In [4]:
LIQUID_hist = pd.read_excel('../data/LIQUID_hist.xlsx', header=None, names=['A', 'B', 'C', 'D'])
LIQUID_rates = pd.read_excel('../data/LIQUID_rates.xlsx', header=None, names=['A', 'B', 'C', 'D'])

## парсер для чтения файла

In [5]:
sector = {}
for i in range(205):
    sector[LIQUID_rates.iloc[i*205]['B'].split(' ')[0]] = list(map(float, LIQUID_rates.iloc[11 + i * 205: 11 + i * 205+ 193, 3]))

sectors_data = pd.DataFrame().from_dict(sector)
calcul_data = LIQUID_rates.iloc[11:11 + 193,:3].rename({'A': 'Date', 'B': 'Time Elapsed Years', 'C':'Time Elapsed Days'}, axis=1).reset_index().drop('index', axis=1)
calcul_data = pd.concat([calcul_data, sectors_data], axis=1)

In [6]:
real_data = LIQUID_hist.iloc[11:11 + 193,:].rename({'A': 'Date', 'B': 'Time Elapsed Years', 
                                                    'C':'Time Elapsed Days', 'D':'LPTH'}, axis=1).reset_index().drop('index', axis=1)

## Поиск 10 похожих замеров на исторический


In [13]:
mse_dict = {}
for i in range(0,205):
    mse_dict[i+3]=(mean_squared_error(real_data.iloc[:,3],calcul_data.iloc[:,i+3]))
mse_dict = {k: v for k, v in sorted(mse_dict.items(), key=lambda item: item[1])}
index  = list(mse_dict.keys())[:10]

In [15]:
index

[138, 58, 163, 83, 74, 188, 105, 159, 73, 168]

## График, где 10 похожих на историческую кривую выделены более ярко, чем все остальные кривые

In [22]:
fig_all = go.Figure()
for i in range(0,205):
    if i in index:
        y_trace = calcul_data.iloc[:,i]
        fig_all.add_trace(go.Scatter(x = real_data.Date, y=y_trace, name=f's{795+i}', opacity = 0.8))
    else:
        y_trace = calcul_data.iloc[:,i+3]
        fig_all.add_trace(go.Scatter(x = real_data.Date, y=y_trace, name=f's{795+i}', opacity = 0.1))
fig_all.add_trace(go.Bar(x = calcul_data.Date, y=real_data.iloc[:,3], name='real SECTOR_17_09_795', opacity = 0.8))

fig_all.update_layout(
    xaxis_title="Date",
    yaxis_title="PTH, sm3",
    legend_title="Sector's number",
    title={
        'text': "LPT of all sectors and LPTH for SECTOR_17_09_795",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig_all.show()

## График с исторической кривой и 10 похожими

In [21]:
fig_all = go.Figure()
for i in index:
    y_trace = calcul_data.iloc[:,i]
    fig_all.add_trace(go.Scatter(x = real_data.Date, y=y_trace, name=f's{795+i}', opacity = 0.6))
fig_all.add_trace(go.Bar(x = calcul_data.Date, y=real_data.iloc[:,3], name='real SECTOR_17_09_795', opacity = 0.8))

fig_all.update_layout(
    xaxis_title="Date",
    yaxis_title="PTH, sm3",
    legend_title="Sector's number",
    title={
        'text': "LPT of all sectors and LPTH for SECTOR_17_09_795",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig_all.show()